In [4]:

import json
import os
import polars as pl
from polars import col as c

from smallflex_data_schema import SmallflexInputSchema
from plotly.subplots import make_subplots

from datetime import date
import plotly.graph_objs as go

from config import settings
import plotly.express as px
from itertools import product
import tqdm

from general_function import pl_to_dict, duckdb_to_dict
from data_display.baseline_plots import plot_first_stage_result, plot_second_stage_result
from data_display.input_data_plots import (
    plot_market_prices, plot_discharge_flow, plot_power, plot_basin_height, plot_basin_height_volume_table,
    plot_continuous_performance_table, plot_discrete_performance_table
)

os.chdir(os.getcwd().replace("/src", ""))


In [12]:
file_names = [
    ".cache/output/baseline/2020_year_results.duckdb",
    ".cache/output/baseline/2021_year_results.duckdb",
    ".cache/output/baseline/2022_year_results.duckdb",
    ".cache/output/baseline/2023_year_results.duckdb"
    
]
for file_name in file_names:
    data = duckdb_to_dict(file_name)
    print(data["income_result"])

Read and validate tables from 2020_year_results.duckdb file: 100%|█████████████████████████████████████████████████████| 4/4 [00:00<00:00, 449.47it/s]


shape: (3, 3)
┌──────────────┬──────────────┬──────────────┐
│ model_0_mask ┆ model_1_mask ┆ model_2_mask │
│ ---          ┆ ---          ┆ ---          │
│ f64          ┆ f64          ┆ f64          │
╞══════════════╪══════════════╪══════════════╡
│ 1.074        ┆ null         ┆ null         │
│ null         ┆ 1.177        ┆ null         │
│ null         ┆ null         ┆ 1.351        │
└──────────────┴──────────────┴──────────────┘


Read and validate tables from 2021_year_results.duckdb file: 100%|█████████████████████████████████████████████████████| 4/4 [00:00<00:00, 431.77it/s]


shape: (3, 3)
┌──────────────┬──────────────┬──────────────┐
│ model_0_mask ┆ model_1_mask ┆ model_2_mask │
│ ---          ┆ ---          ┆ ---          │
│ f64          ┆ f64          ┆ f64          │
╞══════════════╪══════════════╪══════════════╡
│ 3.456        ┆ null         ┆ null         │
│ null         ┆ 3.61         ┆ null         │
│ null         ┆ null         ┆ 4.084        │
└──────────────┴──────────────┴──────────────┘


Read and validate tables from 2022_year_results.duckdb file: 100%|█████████████████████████████████████████████████████| 4/4 [00:00<00:00, 442.06it/s]


shape: (3, 3)
┌──────────────┬──────────────┬──────────────┐
│ model_0_mask ┆ model_1_mask ┆ model_2_mask │
│ ---          ┆ ---          ┆ ---          │
│ f64          ┆ f64          ┆ f64          │
╞══════════════╪══════════════╪══════════════╡
│ 8.225        ┆ null         ┆ null         │
│ null         ┆ 8.843        ┆ null         │
│ null         ┆ null         ┆ 10.18        │
└──────────────┴──────────────┴──────────────┘


Read and validate tables from 2023_year_results.duckdb file: 100%|█████████████████████████████████████████████████████| 4/4 [00:00<00:00, 673.51it/s]

shape: (3, 3)
┌──────────────┬──────────────┬──────────────┐
│ model_0_mask ┆ model_1_mask ┆ model_2_mask │
│ ---          ┆ ---          ┆ ---          │
│ f64          ┆ f64          ┆ f64          │
╞══════════════╪══════════════╪══════════════╡
│ 2.801        ┆ null         ┆ null         │
│ null         ┆ 3.199        ┆ null         │
│ null         ┆ null         ┆ 3.603        │
└──────────────┴──────────────┴──────────────┘


model_0_mask,model_1_mask,model_2_mask
f64,f64,f64
2.801,null,null
null,3.199,null
null,null,3.603


In [9]:


combined_results = data["model_2_mask"]
fig = plot_second_stage_result(
        simulation_results=combined_results, time_divider=7*24
    )
fig.update_layout(dict(title_text="<b>2023 year bench mark optimization results for Aegina hydro power plant<b>", title_x=0.5) )